In [ ]:
import sys
!cp ../input/rapids/rapids.0.13.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.6/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.6"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/

In [ ]:
import gc
import numpy as np
import pandas as pd
import cudf
import cupy as cp
from cuml.neighbors import KNeighborsRegressor
from cuml import SVR
from cuml.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from cuml.metrics import mean_absolute_error, mean_squared_error
from cuml.linear_model import MBSGDRegressor as cumlMBSGDRegressor
from sklearn.linear_model import LinearRegression,LassoLarsCV,ElasticNetCV
from cuml.solvers import SGD as cumlSGD
from cuml.neighbors import KNeighborsRegressor
from sklearn.ensemble import BaggingRegressor

import warnings

import xgboost as xgb
#import lightgbm as lgb

def metric(y_true, y_pred):
    return np.mean(np.sum(np.abs(y_true - y_pred), axis=0)/np.sum(y_true, axis=0))

In [ ]:
fnc_df = cudf.read_csv("../input/trends-assessment-prediction/fnc.csv")
loading_df = cudf.read_csv("../input/trends-assessment-prediction/loading.csv")
labels_df = cudf.read_csv("../input/trends-assessment-prediction/train_scores.csv")

fnc_features, loading_features = list(fnc_df.columns[1:]), list(loading_df.columns[1:])
df = fnc_df.merge(loading_df, on="Id")
labels_df["is_train"] = True

df = df.merge(labels_df, on="Id", how="left")

test_df = df[df["is_train"] != True].copy()
df = df[df["is_train"] == True].copy()

test_df_load_region=test_df.copy() 
df_train=df.copy()
df_scaled=df.copy()
test_df_scaled=test_df.copy()
test_df_load_fnc_loadfnc_region=test_df.copy()
test_df_load_fnc_loadfnc=test_df.copy()


test_df_scaled_rf=test_df.copy()
test_df_load_fnc_loadfnc_region_rf=test_df.copy()
test_df_load_fnc_loadfnc_rf=test_df.copy()

test_df_ensemble=test_df.copy()

# Giving less importance to FNC features since they are easier to overfit due to high dimensionality.
FNC_SCALE = 1/500

df_scaled[fnc_features] *= FNC_SCALE
test_df_scaled[fnc_features] *= FNC_SCALE

In [ ]:
train_sm_data=cudf.read_csv('../input/trends-region400-pca100/train_region_pca100.csv')
test_sm_data=cudf.read_csv('../input/trends-region400-pca100/test_region_pca100.csv',names=train_sm_data.columns,header=0)
region_df = train_sm_data.merge(labels_df, on="Id", how="left")
region_test_df=test_sm_data.copy()
region_features=list(region_test_df.columns[1:])

In [ ]:
blend_weights = {"age":          [0.65, 0.35],
                 "domain1_var1": [0.55, 0.45],
                 "domain1_var2": [0.5,  0.5],
                 "domain2_var1": [0.45, 0.55],
                 "domain2_var2": [0.4,  0.6]}

In [ ]:
%%time

warnings.filterwarnings("ignore", message="Expected column")

targets=['age','domain1_var1','domain1_var2','domain2_var1','domain2_var2']

NUM_FOLDS =7
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=10)

features = loading_features + fnc_features


#rf
overal_score_rf_region = 0
overal_score_rf_fnc = 0
overal_score_rf_load= 0
overal_score_rf_load_fnc =0
overal_score_rf_load_region =0
overal_score_rf_load_fnc_region= 0

overal_score_rf_loadfnc= 0
overal_score_rf_load_fnc_loadfnc =0
overal_score_rf_loadfnc_region=0
overal_score_rf_load_fnc_loadfnc_region =0
#svr
overal_score_svr_region = 0
overal_score_svr_fnc = 0
overal_score_svr_load= 0
overal_score_svr_load_fnc =0
overal_score_svr_load_region =0
overal_score_svr_load_fnc_region= 0

overal_score_svr_loadfnc= 0
overal_score_svr_load_fnc_loadfnc =0
overal_score_svr_loadfnc_region=0
overal_score_svr_load_fnc_loadfnc_region =0



overal_score_ensemble=0

for target, c,b,w,a,r in [("age", 64,20, 0.3,0.00005,0.5), ("domain1_var1",12,10, 0.175,0.00001,0.2), ("domain1_var2",16, 10, 0.175,0.15,0.5), ("domain2_var1",18,10, 0.175,0.0001,0.9), ("domain2_var2",10,10,0.175,0.0001,0.1)]:     
    
    
    #rf
    y_val_rf_region = np.zeros(region_df.shape[0])
    y_val_rf_load = np.zeros(df.shape[0])
    y_val_rf_fnc = np.zeros(df.shape[0])
    y_val_rf_load_fnc= np.zeros(df.shape[0])
    y_val_rf_load_region= np.zeros(df.shape[0])
    y_val_rf_load_fnc_region= np.zeros(df.shape[0])
     
    y_val_rf_loadfnc = np.zeros(df.shape[0])
    y_val_rf_load_fnc_loadfnc= np.zeros(df.shape[0])
    y_val_rf_loadfnc_region= np.zeros(df.shape[0])
    y_val_rf_load_fnc_loadfnc_region= np.zeros(df.shape[0])


    y_test_rf_region = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_rf_load_fnc = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_rf_load_region = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_rf_load_fnc_region = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    
    y_test_rf_region_loadfnc = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_rf_load_fnc_loadfnc = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_rf_load_fnc_loadfnc_region = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    #svr
    y_val_svr_region = np.zeros(region_df.shape[0])
    y_val_svr_load = np.zeros(df.shape[0])
    y_val_svr_fnc = np.zeros(df.shape[0])
    y_val_svr_load_fnc= np.zeros(df.shape[0])
    y_val_svr_load_region= np.zeros(df.shape[0])
    y_val_svr_load_fnc_region= np.zeros(df.shape[0])
     
    y_val_svr_loadfnc = np.zeros(df.shape[0])
    y_val_svr_load_fnc_loadfnc= np.zeros(df.shape[0])
    y_val_svr_loadfnc_region= np.zeros(df.shape[0])
    y_val_svr_load_fnc_loadfnc_region= np.zeros(df.shape[0])


    y_test_svr_region = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_svr_load_fnc = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_svr_load_region = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_svr_load_fnc_region = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    
    y_test_svr_region_loadfnc = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_svr_load_fnc_loadfnc = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    y_test_svr_load_fnc_loadfnc_region = np.zeros((region_test_df.shape[0], NUM_FOLDS))
    
   
   
    #ensemble
    y_val_ensemble=np.zeros(df.shape[0])
    y_test_ensemble= np.zeros((region_test_df.shape[0], NUM_FOLDS))
    
    for f, (train_ind, val_ind) in enumerate(kf.split(df, df)):
        region_train_df, region_val_df = region_df.iloc[train_ind], region_df.iloc[val_ind]
        region_train_df = region_train_df[region_train_df[target].notnull()]
        
        train_df, val_df =df.iloc[train_ind],df.iloc[val_ind]
        train_df =train_df[train_df[target].notnull()]
        
        train_df_scaled, val_df_scaled = df_scaled.iloc[train_ind], df_scaled.iloc[val_ind]
        train_df_scaled = train_df_scaled[train_df_scaled[target].notnull()]
        
        #rf 
        #--------------------------------------------------------------------------------------
        rf_1 =  BaggingRegressor(Ridge(alpha = 0.0001), n_estimators=100, random_state=42, max_samples=0.3, max_features=0.3)
        rf_1.fit(train_df.to_pandas()[fnc_features].values, train_df.to_pandas()[target].values)
        rf_2 = BaggingRegressor(Ridge(alpha = 0.0001), n_estimators=100, random_state=2, max_samples=0.3, max_features=0.6)
        rf_2.fit(train_df.to_pandas()[loading_features].values, train_df.to_pandas()[target].values)
        rf_3= BaggingRegressor(Ridge(alpha = 0.0001), n_estimators=100, random_state=4, max_samples=0.3, max_features=0.3)
        rf_3.fit(region_train_df.to_pandas()[region_features].values, region_train_df.to_pandas()[target].values)
        rf_4= BaggingRegressor(Ridge(alpha = 0.0001),n_estimators=100, random_state=52, max_samples=0.3, max_features=0.3)
        rf_4.fit(train_df_scaled.to_pandas()[features].values, train_df_scaled.to_pandas()[target].values)
        
               
        n=val_ind
        y_val_rf_fnc[n]= cudf.from_pandas(pd.Series(rf_1.predict(val_df.to_pandas()[fnc_features])))
        y_val_rf_load[n]= cudf.from_pandas(pd.Series(rf_2.predict(val_df.to_pandas()[loading_features])))
        y_val_rf_region[n]= cudf.from_pandas(pd.Series(rf_3.predict(region_val_df.to_pandas()[region_features])))
        y_val_rf_load_fnc[n]= (y_val_rf_load[n]+y_val_rf_fnc[n])/2
        y_val_rf_load_region[n]= (y_val_rf_load[n]+y_val_rf_region[n])/2
        y_val_rf_load_fnc_region[n]=(y_val_rf_load[n]+y_val_rf_fnc[n]+y_val_rf_region[n])/3
        
        
        
        y_val_rf_loadfnc[n] =  cudf.from_pandas(pd.Series(rf_4.predict(val_df_scaled.to_pandas()[features])))
        y_val_rf_load_fnc_loadfnc[n]= (y_val_rf_fnc[n]+y_val_rf_load[n]+y_val_rf_loadfnc[n])/3
        y_val_rf_loadfnc_region[n]=  y_val_rf_loadfnc[n]*0.7+y_val_rf_region[n]*0.3
        y_val_rf_load_fnc_loadfnc_region[n]= (y_val_rf_load[n]+y_val_rf_fnc[n]+y_val_rf_region[n]+y_val_rf_loadfnc[n])/4
        
        
        test_pred_1 =  cudf.from_pandas(pd.Series(rf_1.predict(test_df.to_pandas()[fnc_features])))
        test_pred_2 =  cudf.from_pandas(pd.Series(rf_2.predict(test_df.to_pandas()[loading_features])))
        test_pred_3 =  cudf.from_pandas(pd.Series(rf_3.predict(region_test_df.to_pandas()[region_features])))
        test_pred_4 =  cudf.from_pandas(pd.Series(rf_4.predict(test_df_scaled.to_pandas()[features])))
        
        test_pred_rf_load_fnc = (test_pred_1+test_pred_2)/2
        test_pred_rf_load_fnc = cp.asnumpy(test_pred_rf_load_fnc.values.flatten())
        test_pred_rf_load_region = (test_pred_1+test_pred_3)/2
        test_pred_rf_load_region = cp.asnumpy(test_pred_rf_load_region.values.flatten())
        test_pred_rf_load_fnc_region = (test_pred_1+test_pred_2+test_pred_3)/3
        test_pred_rf_load_fnc_region = cp.asnumpy(test_pred_rf_load_fnc_region.values.flatten()) 
        test_pred_rf_region = cp.asnumpy(test_pred_3.values.flatten())
        
        test_pred_rf_region_loadfnc=test_pred_3*0.3+test_pred_4*0.7
        test_pred_rf_region_loadfnc=cp.asnumpy(test_pred_rf_region_loadfnc.values.flatten())
        test_pred_rf_load_fnc_loadfnc=(test_pred_1+test_pred_2+test_pred_4)/3
        test_pred_rf_load_fnc_loadfnc=cp.asnumpy(test_pred_rf_load_fnc_loadfnc.values.flatten())
        test_pred_rf_load_fnc_loadfnc_region=(test_pred_1+test_pred_2+test_pred_3+test_pred_4)/4
        test_pred_rf_load_fnc_loadfnc_region=cp.asnumpy(test_pred_rf_load_fnc_loadfnc_region.values.flatten())

        y_test_rf_load_fnc[:, f] = test_pred_rf_load_fnc
        y_test_rf_load_region[:, f] = test_pred_rf_load_region
        y_test_rf_load_fnc_region[:, f] = test_pred_rf_load_fnc_region
        y_test_rf_region[:, f] = test_pred_rf_region
        
        y_test_rf_region_loadfnc[:, f]  =test_pred_rf_region_loadfnc
        y_test_rf_load_fnc_loadfnc[:, f]  = test_pred_rf_load_fnc_loadfnc
        y_test_rf_load_fnc_loadfnc_region[:, f] =test_pred_rf_load_fnc_loadfnc_region
        

        #svr 
        #--------------------------------------------------------------------------------------
        svr_1 = SVR(C=c, cache_size=3000.0)
        svr_1.fit(train_df[fnc_features], train_df[target])
        svr_2 =SVR(C=c, cache_size=1000.0)
        svr_2.fit(train_df[loading_features], train_df[target])
        svr_3= SVR(C=b, cache_size=5000.0)
        svr_3.fit(region_train_df[region_features], region_train_df[target])
        svr_4= SVR(C=c, cache_size=4000.0)
        svr_4.fit(train_df_scaled[features], train_df_scaled[target])
        
        
        
        n=val_ind
        y_val_svr_fnc[n]=svr_1.predict(val_df[fnc_features])
        y_val_svr_load[n]=svr_2.predict(val_df[loading_features])
        y_val_svr_region[n]=svr_3.predict(region_val_df[region_features])
        y_val_svr_load_fnc[n]= (y_val_svr_load[n]+y_val_svr_fnc[n])/2
        y_val_svr_load_region[n]= (y_val_svr_load[n]+y_val_svr_region[n])/2
        y_val_svr_load_fnc_region[n]=(y_val_svr_load[n]+y_val_svr_fnc[n]+y_val_svr_region[n])/3
        
        y_val_svr_loadfnc[n] = svr_4.predict(val_df_scaled[features])
        y_val_svr_load_fnc_loadfnc[n]= (y_val_svr_fnc[n]+y_val_svr_load[n]+y_val_svr_loadfnc[n])/3
        y_val_svr_loadfnc_region[n]=  y_val_svr_loadfnc[n]*0.7+y_val_svr_region[n]*0.3
        y_val_svr_load_fnc_loadfnc_region[n]= (y_val_svr_load[n]+y_val_svr_fnc[n]+y_val_svr_region[n]+y_val_svr_loadfnc[n])/4

        
        test_pred_1 = svr_1.predict(test_df[fnc_features])
        test_pred_2 = svr_2.predict(test_df[loading_features])
        test_pred_3 = svr_3.predict(region_test_df[region_features])
        test_pred_4 = svr_4.predict(test_df_scaled[features])
        
        test_pred_svr_load_fnc = (test_pred_1+test_pred_2)/2
        test_pred_svr_load_fnc = cp.asnumpy(test_pred_svr_load_fnc.values.flatten())
        test_pred_svr_load_region = (test_pred_1+test_pred_3)/2
        test_pred_svr_load_region = cp.asnumpy(test_pred_svr_load_region.values.flatten())
        test_pred_svr_load_fnc_region = (test_pred_1+test_pred_2+test_pred_3)/3
        test_pred_svr_load_fnc_region = cp.asnumpy(test_pred_svr_load_fnc_region.values.flatten()) 
        test_pred_svr_region = cp.asnumpy(test_pred_3.values.flatten())
        
        test_pred_svr_region_loadfnc=test_pred_3*0.3+test_pred_4*0.7
        test_pred_svr_region_loadfnc=cp.asnumpy(test_pred_svr_region_loadfnc.values.flatten())
        test_pred_svr_load_fnc_loadfnc=(test_pred_1+test_pred_2+test_pred_4)/3
        test_pred_svr_load_fnc_loadfnc=cp.asnumpy(test_pred_svr_load_fnc_loadfnc.values.flatten())
        test_pred_svr_load_fnc_loadfnc_region=(test_pred_1+test_pred_2+test_pred_3+test_pred_4)/4
        test_pred_svr_load_fnc_loadfnc_region=cp.asnumpy(test_pred_svr_load_fnc_loadfnc_region.values.flatten())

        y_test_svr_load_fnc[:, f] = test_pred_svr_load_fnc
        y_test_svr_load_region[:, f] = test_pred_svr_load_region
        y_test_svr_load_fnc_region[:, f] = test_pred_svr_load_fnc_region
        y_test_svr_region[:, f] = test_pred_svr_region
        
        y_test_svr_region_loadfnc[:, f]  =test_pred_svr_region_loadfnc
        y_test_svr_load_fnc_loadfnc[:, f]  = test_pred_svr_load_fnc_loadfnc
        y_test_svr_load_fnc_loadfnc_region[:, f] =test_pred_svr_load_fnc_loadfnc_region
               

        #ensemble
        #------------------------------------------------
        if target is "age":
            y_val_ensemble[n] = blend_weights[target][0]*y_val_svr_loadfnc[n] \
                              + blend_weights[target][1]*y_val_rf_loadfnc[n]
            y_test_ensemble[:,f]= blend_weights[target][0]*cp.asnumpy(svr_4.predict(test_df_scaled[features]).values.flatten()) \
                                + blend_weights[target][1]*cp.asnumpy(cudf.from_pandas(pd.Series(rf_4.predict(test_df_scaled.to_pandas()[features]))))
            print('ensemble_test')
        elif target is "domain1_var1":
            y_val_ensemble[n] = blend_weights[target][0]*y_val_svr_load_fnc[n] \
                              + blend_weights[target][1]*y_val_rf_loadfnc[n]
            y_test_ensemble[:,f]= blend_weights[target][0]*test_pred_svr_load_fnc \
                                + blend_weights[target][1]*cp.asnumpy(cudf.from_pandas(pd.Series(rf_4.predict(test_df_scaled.to_pandas()[features]))))
        elif target is 'domain1_var2':
            y_val_ensemble[n] = blend_weights[target][0]*y_val_svr_load_fnc[n] \
                              + blend_weights[target][1]*y_val_rf_loadfnc[n]
            y_test_ensemble[:,f]= blend_weights[target][0]*test_pred_svr_load_fnc \
                                + blend_weights[target][1]*cp.asnumpy(cudf.from_pandas(pd.Series(rf_4.predict(test_df_scaled.to_pandas()[features]))))
        elif target is 'domain2_var1':
            y_val_ensemble[n] = blend_weights[target][0]*y_val_svr_load_fnc[n] \
                              + blend_weights[target][1]*y_val_rf_loadfnc[n]
            y_test_ensemble[:,f]= blend_weights[target][0]*test_pred_svr_load_fnc \
                                + blend_weights[target][1]*cp.asnumpy(cudf.from_pandas(pd.Series(rf_4.predict(test_df_scaled.to_pandas()[features]))))
        elif target is 'domain2_var2':
            y_val_ensemble[n] = blend_weights[target][0]*y_val_svr_load_fnc[n] \
                              + blend_weights[target][1]*y_val_rf_loadfnc[n]
            y_test_ensemble[:,f]= blend_weights[target][0]*test_pred_svr_load_fnc \
                                + blend_weights[target][1]*cp.asnumpy(cudf.from_pandas(pd.Series(rf_4.predict(test_df_scaled.to_pandas()[features]))))
    #rf
    df["pred_rf_load_{}".format(target)] = y_val_rf_load
    df["pred_rf_fnc_{}".format(target)] = y_val_rf_fnc
    region_df["pred_rf_region_{}".format(target)] = y_val_rf_region  
    df["pred_rf_load_fnc_{}".format(target)] = y_val_rf_load_fnc
    df["pred_rf_load_region_{}".format(target)] = y_val_rf_load_region
    df["pred_rf_load_fnc_region_{}".format(target)] = y_val_rf_load_fnc_region
        
    df["pred_rf_loadfnc_{}".format(target)] = y_val_rf_loadfnc
    df["pred_rf_loadfnc_region_{}".format(target)] = y_val_rf_loadfnc_region
    df["pred_rf_load_fnc_loadfnc_{}".format(target)] = y_val_rf_load_fnc_loadfnc
    df["pred_rf_load_fnc_loadfnc_region_{}".format(target)] = y_val_rf_load_fnc_loadfnc_region
        
    region_test_df[target] = y_test_rf_region.mean(axis=1)
    test_df_load_region[target]=y_test_rf_load_region.mean(axis=1)
    test_df[target] = y_test_rf_load_fnc_region.mean(axis=1)
    test_df_scaled[target]=y_test_rf_region_loadfnc.mean(axis=1)
    test_df_load_fnc_loadfnc[target]=y_test_rf_load_fnc_loadfnc.mean(axis=1)
    test_df_load_fnc_loadfnc_region[target]=y_test_rf_load_fnc_loadfnc_region.mean(axis=1)
    
    
    score_rf_load= metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_load_{}".format(target)].values)
    score_rf_fnc= metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_fnc_{}".format(target)].values)
    score_rf_region = metric(region_df[region_df[target].notnull()][target].values, region_df[region_df[target].notnull()]["pred_rf_region_{}".format(target)].values)
    score_rf_load_fnc = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_load_fnc_{}".format(target)].values)      
    score_rf_load_region = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_load_region_{}".format(target)].values)   
    score_rf_load_fnc_region = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_load_fnc_region_{}".format(target)].values)   
    
    score_rf_loadfnc = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_loadfnc_{}".format(target)].values)      
    score_rf_loadfnc_region = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_loadfnc_region_{}".format(target)].values)   
    score_rf_load_fnc_loadfnc = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_load_fnc_loadfnc_{}".format(target)].values)   
    score_rf_load_fnc_loadfnc_region = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_rf_load_fnc_loadfnc_region_{}".format(target)].values)   

   
    overal_score_rf_load += w*score_rf_load
    overal_score_rf_fnc += w*score_rf_fnc
    overal_score_rf_region += w*score_rf_region
    overal_score_rf_load_fnc += w*score_rf_load_fnc
    overal_score_rf_load_region += w*score_rf_load_region
    overal_score_rf_load_fnc_region += w*score_rf_load_fnc_region
    
    overal_score_rf_loadfnc += w*score_rf_loadfnc
    overal_score_rf_load_fnc_loadfnc += w*score_rf_load_fnc_loadfnc
    overal_score_rf_loadfnc_region += w*score_rf_loadfnc_region
    overal_score_rf_load_fnc_loadfnc_region  += w*score_rf_load_fnc_loadfnc_region


    print(target)
    print("--"*20)
    print('rf: Load                   |',target, np.round(score_rf_load, 8))
    print('rf: FNC                    |',target, np.round(score_rf_fnc, 8))
    print('rf: Region                 |',target, np.round(score_rf_region, 8))
    print('rf: Loadfnc                |',target, np.round(score_rf_loadfnc, 8))
    print('rf: Load+FNC               |',target, np.round(score_rf_load_fnc, 8))
    print('rf: Load+Region            |',target, np.round(score_rf_load_region, 8))
    print('rf: Load+FNC+Region        |',target, np.round(score_rf_load_fnc_region, 8))
    print('rf: Load+FNC+Loadfnc       |',target, np.round(score_rf_load_fnc_loadfnc, 8))
    print('rf: Loadfnc+Region         |',target, np.round(score_rf_loadfnc_region, 8))
    print('rf: Load+FNC+Loadfnc+Region|',target, np.round(score_rf_load_fnc_loadfnc_region, 8))
    print("--"*30)
 
    #svr
    df["pred_svr_load_{}".format(target)] = y_val_svr_load
    df["pred_svr_fnc_{}".format(target)] = y_val_svr_fnc
    region_df["pred_svr_region_{}".format(target)] = y_val_svr_region  
    df["pred_svr_load_fnc_{}".format(target)] = y_val_svr_load_fnc
    df["pred_svr_load_region_{}".format(target)] = y_val_svr_load_region
    df["pred_svr_load_fnc_region_{}".format(target)] = y_val_svr_load_fnc_region
        
    df["pred_svr_loadfnc_{}".format(target)] = y_val_svr_loadfnc
    df["pred_svr_loadfnc_region_{}".format(target)] = y_val_svr_loadfnc_region
    df["pred_svr_load_fnc_loadfnc_{}".format(target)] = y_val_svr_load_fnc_loadfnc
    df["pred_svr_load_fnc_loadfnc_region_{}".format(target)] = y_val_svr_load_fnc_loadfnc_region
    
    
    region_test_df[target] = y_test_svr_region.mean(axis=1)
    test_df_load_region[target]=y_test_svr_load_region.mean(axis=1)
    test_df[target] = y_test_svr_load_fnc_region.mean(axis=1)
    test_df_scaled[target]=y_test_svr_region_loadfnc.mean(axis=1)
    test_df_load_fnc_loadfnc[target]=y_test_svr_load_fnc_loadfnc.mean(axis=1)
    test_df_load_fnc_loadfnc_region[target]=y_test_svr_load_fnc_loadfnc_region.mean(axis=1)
    
    
    
    score_svr_load= metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_load_{}".format(target)].values)
    score_svr_fnc= metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_fnc_{}".format(target)].values)
    score_svr_region = metric(region_df[region_df[target].notnull()][target].values, region_df[region_df[target].notnull()]["pred_svr_region_{}".format(target)].values)
    score_svr_load_fnc = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_load_fnc_{}".format(target)].values)      
    score_svr_load_region = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_load_region_{}".format(target)].values)   
    score_svr_load_fnc_region = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_load_fnc_region_{}".format(target)].values)   
    
    score_svr_loadfnc = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_loadfnc_{}".format(target)].values)      
    score_svr_loadfnc_region = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_loadfnc_region_{}".format(target)].values)   
    score_svr_load_fnc_loadfnc = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_load_fnc_loadfnc_{}".format(target)].values)   
    score_svr_load_fnc_loadfnc_region = metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_svr_load_fnc_loadfnc_region_{}".format(target)].values)   

      
    overal_score_svr_load += w*score_svr_load
    overal_score_svr_fnc += w*score_svr_fnc
    overal_score_svr_region += w*score_svr_region
    overal_score_svr_load_fnc += w*score_svr_load_fnc
    overal_score_svr_load_region += w*score_svr_load_region
    overal_score_svr_load_fnc_region += w*score_svr_load_fnc_region
    
    overal_score_svr_loadfnc += w*score_svr_loadfnc
    overal_score_svr_load_fnc_loadfnc += w*score_svr_load_fnc_loadfnc
    overal_score_svr_loadfnc_region += w*score_svr_loadfnc_region
    overal_score_svr_load_fnc_loadfnc_region  += w*score_svr_load_fnc_loadfnc_region

    print(target)
    print("--"*20)
    print('SVR: Load                   |',target, np.round(score_svr_load, 8))
    print('SVR: FNC                    |',target, np.round(score_svr_fnc, 8))
    print('SVR: Region                 |',target, np.round(score_svr_region, 8))
    print('SVR: Loadfnc                |',target, np.round(score_svr_loadfnc, 8))
    print('SVR: Load+FNC               |',target, np.round(score_svr_load_fnc, 8))
    print('SVR: Load+Region            |',target, np.round(score_svr_load_region, 8))
    print('SVR: Load+FNC+Region        |',target, np.round(score_svr_load_fnc_region, 8))
    print('SVR: Load+FNC+Loadfnc       |',target, np.round(score_svr_load_fnc_loadfnc, 8))
    print('SVR: Loadfnc+Region         |',target, np.round(score_svr_loadfnc_region, 8))
    print('SVR: Load+FNC+Loadfnc+Region|',target, np.round(score_svr_load_fnc_loadfnc_region, 8))
    print("--"*30)
  
    print("--"*30)
    
    df["pred_ensemble_{}".format(target)] = y_val_ensemble
    test_df_ensemble[target]=y_test_ensemble.mean(axis=1)
    score_ensemble= metric(df[df[target].notnull()][target].values, df[df[target].notnull()]["pred_ensemble_{}".format(target)].values)
    overal_score_ensemble += w*score_ensemble
    print('Ensemble                    |',target, np.round(score_ensemble, 8))
    
print("Overal score of rf: Load                    |", np.round(overal_score_rf_load, 4))
print("Overal score of rf: FNC                     |", np.round(overal_score_rf_fnc, 4))
print("Overal score of rf: Region                  |", np.round(overal_score_rf_region, 4))
print("Overal score of rf: Loadfnc                 |", np.round(overal_score_rf_loadfnc, 4))
print("Overal score of rf: Load+FNC                |", np.round(overal_score_rf_load_fnc, 4))
print("Overal score of rf: Load+Region             |", np.round(overal_score_rf_load_region, 4))
print("Overal score of rf: Load+FNC+Region         |", np.round(overal_score_rf_load_fnc_region, 4))
print("Overal score of rf: Load+FNC+Loadfnc        |", np.round(overal_score_rf_load_fnc_loadfnc, 4))
print("Overal score of rf: Loadfnc+Region          |", np.round(overal_score_rf_loadfnc_region, 4))
print("Overal score of rf: Load+FNC+Loadfnc+Region |", np.round(overal_score_rf_load_fnc_loadfnc_region, 4))
print("Overal score of SVR: Load                   |", np.round(overal_score_svr_load, 4))
print("Overal score of SVR: FNC                    |", np.round(overal_score_svr_fnc, 4))
print("Overal score of SVR: Region                 |", np.round(overal_score_svr_region, 4))
print("Overal score of SVR: Loadfnc                |", np.round(overal_score_svr_loadfnc, 4))
print("Overal score of SVR: Load+FNC               |", np.round(overal_score_svr_load_fnc, 4))
print("Overal score of SVR: Load+Region            |", np.round(overal_score_svr_load_region, 4))
print("Overal score of SVR: Load+FNC+Region        |", np.round(overal_score_svr_load_fnc_region, 4))
print("Overal score of SVR: Load+FNC+Loadfnc       |", np.round(overal_score_svr_load_fnc_loadfnc, 4))
print("Overal score of SVR: Loadfnc+Region         |", np.round(overal_score_svr_loadfnc_region, 4))
print("Overal score of SVR: Load+FNC+Loadfnc+Region|", np.round(overal_score_svr_load_fnc_loadfnc_region, 4))
print("--"*30)
print("--"*30)
print("Overal score of Ensemble                    |", np.round(overal_score_ensemble, 4))

In [ ]:
sub_df = cudf.melt(test_df_ensemble[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.to_csv("submission_rapids_ensemble_rf_svr.csv", index=False)
sub_df.head(10)

In [ ]:
sub_df = cudf.melt(region_test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.to_csv("submission_rapids_region400_svr.csv", index=False)
sub_df.head(10)

In [ ]:
sub_df = cudf.melt(test_df_load_region[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.to_csv("submission_rapids_load_region400_svr.csv", index=False)
sub_df.head(10)

In [ ]:
sub_df = cudf.melt(test_df[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.to_csv("submission_rapids_load_fnc_region400_svr.csv", index=False)
sub_df.head(10)

In [ ]:
sub_df = cudf.melt(test_df_load_fnc_loadfnc_region_rf[["Id", "age", "domain1_var1", "domain1_var2", "domain2_var1", "domain2_var2"]], id_vars=["Id"], value_name="Predicted")
sub_df["Id"] = sub_df["Id"].astype("str") + "_" +  sub_df["variable"].astype("str")

sub_df = sub_df.drop("variable", axis=1).sort_values("Id")
assert sub_df.shape[0] == test_df.shape[0]*5
sub_df.to_csv("submission_rapids_region400_baggingela.csv", index=False)
sub_df.head(10)

## Log
V11.SVR+KNN+Elasticnet: 0.1599

In [ ]:
import pandas as pd

In [ ]:
df_region1= pd.read_csv("../input/region-submission/submission_rapids_region400_knn_elasticnet_svr.csv")
df_region2= pd.read_csv("../input/region-submission/submission_rapids_region400_knn_elasticnet_svr (1).csv")
df_region3= pd.read_csv("../input/region-submission/submission_rapids_load_fnc_region400_knn_svr.csv")
df_region4= pd.read_csv("../input/nnnnnn/submission_rapids_load_fnc_region400_svr.csv")
df_region1['Predicted']=(df_region1['Predicted']*0.5+df_region2['Predicted']*0.5)*0.6+df_region3['Predicted']*0.2+df_region4['Predicted']*0.2
df_region1.head(10)

In [ ]:
df_ela1 = pd.read_csv('../input/elasticnetcv2-submission/400region_load_FNC_ElasticNetCV.csv')
df_ela2 = pd.read_csv("../input/bayesoptela1/bayes_opt_loadfnc_skfold10_pure_elasticnetcv.csv")
df_ela3 = pd.read_csv("../input/full-skf10-elasticnet-0158/loadfnc_full_skfold10_elasticnetcv.csv")
df_ela4 = pd.read_csv("../input/full-skf10-elasticnet-0158/loadfnc_skfold20_pure_elasticnetcv_py_val0.1582.csv")
df_ela5 = pd.read_csv("../input/full-skf10-elasticnet-0158/Nodropoutliers_loadfnc_skfold20(Noshuffle)_pure_elasticnetcv_fncscale.csv")
df_ela6 = pd.read_csv("../input/dsdwe3/loadfnc_skfold15_elasticnetcv_val0.1579.csv")
df_ela1['Predicted']=(df_ela1['Predicted']*1+df_ela2['Predicted']+df_ela3['Predicted']+df_ela4['Predicted']+df_ela5['Predicted']+df_ela6['Predicted'])/6
df_ela1.head(10)

In [ ]:
df_svr1=pd.read_csv("../input/dsdwe3/submission_rapids_ensemble.csv")
df_svr2=pd.read_csv("../input/rapid-svr/submission_rapids_ensemble_SVR_val0.1581.csv")
df_svr3=pd.read_csv("../input/rapid-svr/submission_rapids_ensemble_SVR_KNN.csv")
df_svr4=pd.read_csv("../input/elasticnet1/submission_rapids_ensemble_6SVR_4KNN.csv")
#df_svr5=pd.read_csv("../input/elasticnet1/submission_svr2_ela4_ensemble (1).csv")
df_svr6=pd.read_csv('../input/elasticnetcv2-submission/400region_load_FNC_SVR.csv')
df_svr1['Predicted']=df_svr1['Predicted']*0.4+df_svr2['Predicted']*0.1+df_svr3['Predicted']*0.3+df_svr4['Predicted']*0.15+df_svr6['Predicted']*0.05
df_svr1.head(15)

In [ ]:
df_ela_svr1=pd.read_csv('../input/elasticnetcv2-submission/400region_fold5_ElasticnetCV_SVR.csv')
df_ela_svr2=pd.read_csv('../input/elasticnetcv2-submission/400region_load_FNC_ElasticNetCV_SVR.csv')
df_ela_svr1['Predicted']=df_ela_svr1['Predicted']*0.5+df_ela_svr2['Predicted']*0.5

In [ ]:
df_svr1['Predicted']=df_ela1['Predicted']*0.3+df_svr1['Predicted']*0.4+df_region1['Predicted']*0.25+df_ela_svr1['Predicted']*0.05
df_mm=pd.read_csv("../input/elasticnet1/submission_svr2_ela4_ensemble (1).csv")
df_svr1['Predicted']=df_svr1['Predicted']*0.8+df_mm['Predicted']*0.2
df_svr1.head(15)

In [ ]:
df_svr1.to_csv("submission_ramdom_v12.csv", index=False)

### Log
> * V6: KNN+SVR+elasticnet 
> * V8: KNN+SVR submission_random_v8.csv LB 0.1601 
> * V9: SVR (load+fnc+region(full features))
> * V10: SVR (load+fnc+loadfnc+region(full features))

In [ ]:
#df_ela1.to_csv("submission_svr_elasticnetcv2_load_fnc_region.csv", index=False)